# **A Predictive Model for Suicidal Ideation of Elderly Using Random Forests Algorithm**

<br/>

📍 개발 단계 ( Development Step )

1. 데이터 구조 훑어보기
2. 테스트 세트 만들기
3. 머신러닝 알고리즘을 위해 데이터를 준비하기
4. 모델을 훈련시키고, 교차 검증과 랜덤 서치를 통해 모델을 조정하기
5. 오차 행렬, f1 스코어, 정확도, 재현율, 정밀도, 특이도 계산하기
6. 변수중요도 산출하기
7. ROC-AUC 커브 그리고, AUC 점수 계산하기

<br/>

📍 보건복지부가 실시한 노인실태조사 마이크로데이터 3개년치를 활용함

- 2020
- 2017
- 2014

( 출처: 보건복지데이터포털 https://data.kihasa.re.kr/kihasa/main.html )

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/[...]/2022-ML1-project

## (1) 데이터 구조 훑어보기

### 최종 데이터셋 불러오기

In [ ]:
import pandas as pd

In [ ]:
filename = '/content/drive/MyDrive/[...]/2022-ML1-project/data/elderly_final_dataset.csv'

elderly = pd.read_csv(filename, index_col=0)

elderly

In [ ]:
elderly.index.value_counts()

### info() 메서드를 통해서 데이터 타입 파악하고, 타입 변경하기

In [ ]:
elderly.info()

1. 변수 타입 변경하기

In [ ]:
elderly = elderly.astype({'총 가구원 수':'category',
                          '노인(만65세 이상) 가구원 수':'category',
                          '노인가구형태':'category',
                          '주관적 건강상태':'category',
                          '의사진단 만성질환 총 수':'category',
                          '현재 흡연 여부':'category',
                          '평소 음주 여부':'category',
                          '평소 운동 여부':'category',
                          '지난 2년 간 건강검진 여부(치매검진 제외)':'category',
                          '일상생활수행에 있어 도움받음 여부':'category',
                          '노인장기요양보험 등급 신청 경험 유무':'category',
                          '종교 여부':'category',
                          '현재 경제활동 여부':'category',
                          '노인일자리 및 사회활동지원사업 참여 경험 유무':'category',
                          '낙상_지난 1년 간 경험 유무':'category',
                          '타인으로부터 신체적 고통을 당함_ 여부':'category',
                          '타인으로 인해 감정을 상함_여부':'category',
                          '타인으로부터 금전적 피해를 입음_여부':'category',
                          '가족이나 보호자가 돌봐주지 않음_여부':'category',
                          '가족이나 보호자의 방임 및 생활비 미지원 여부':'category',
                          '일상생활 차별 경험 유무':'category',
                          '만 60세 이후 자살 생각':'category',
                          '국민기초생활보장수급자 또는 의료급여수급자 여부':'category',
                          '노인 조사 대상자 성별':'category',
                          '노인 조사 대상자 만연령':'category',
                          '우울증척도':'category',
                          '영양관리':'category',
                          '지난 1년 간 여가문화활동 여부':'category',
                          '지난 1년 간 여가문화시설 이용 여부':'category',
                          '거주형태(가구)':'category',
                          '주택 종류':'category',
                          '주거 위치':'category',
                          '노인 조사 대상자 교육수준':'category'})

2. 타입 변경 잘 되었는지 확인하기

In [ ]:
elderly.info()

### value_counts() 메서드를 통해 각 변수의 값들의 분포 파악하기

In [ ]:
elderly_feature_list = elderly.columns.tolist()

for feature in elderly_feature_list:
    print(elderly[feature].value_counts())
    print()

In [ ]:
elderly.describe()

### Histogram으로 수치형 데이터 분포 살펴보기

- 한글 깨짐 해결하기 - 폰트 설정하기

In [ ]:
'''!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf'''

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

!apt-get update -qq
!apt-get install fonts-nanum* -qq

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()
mpl.rcParams['axes.unicode_minus'] = False

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rc

elderly.hist(bins=80, figsize=(20,15))

plt.rc('font', family='NanumBarunGothic')
plt.savefig('Numerical_feature_histogram.png')
plt.show()

## (2) 테스트 세트 만들기

- 데이터를 더 깊게 들여다보기 전에 테스트 세트를 따로 떼어놓아야 함
- 데이터 스누핑(data snooping) 편향: 테스트 세트를 먼저 들여다보면 겉으로 드러난 어떤 패턴에 속아 특정 머신러닝 모델을 선택하게 됨

- 가장 먼저 예측에 활용할 데이터 5개 정도 추출하기

In [ ]:
furture_pred_dataset = elderly.tail(5)

elderly = elderly.drop([i for i in range(len(elderly)-5,len(elderly))])

furture_pred_dataset

In [ ]:
import numpy as np
np.random.seed(42)

- 계층적 샘플링
  - 예를 들어, 소득별로 샘플링을 할때 각 계층의 샘플링수가 충분히 많아야하고, 너무 많은 계층으로 나누면 안된다
  - pd.cut() 함수를 사용해 카테고리 5개를 가진 소득 카테고리 특성을 만듦

- 순수 무작위 샘플링의 문제점
  - 데이터셋이 충분히 크다면(특히 특성 수에 비해) 일반적으로 괜찮지만, 그렇지 않다면 샘플링 편향이 생길 가능성이 큼
  - 테스트 세트가 모집단을 대표할 수 있게끔 각 계층(strata)에서 올바른 수의 샘플을 추출해야 함 -> 계층적 샘플링(stratified sampling)

In [ ]:
elderly["경제상태_만족도_cat"] = pd.cut(elderly["만족도_경제상태"],
                               bins=[0.0, 1.0, 2.0, 3.0, 4.0, 6.0],
                               labels=[1, 2, 3, 4, 5])

In [ ]:
elderly["경제상태_만족도_cat"].hist()

In [ ]:
elderly['경제상태_만족도_cat'].value_counts()

- Stratified ShuffleSplit
  - 만족도_경제상태 카테고리를 기반으로 계층 샘플링

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

for train_index, test_index in split.split(elderly, elderly["경제상태_만족도_cat"]):
    strat_train_set = elderly.loc[train_index]
    strat_test_set = elderly.loc[test_index]

- 비율 확인하기

In [ ]:
strat_train_set["경제상태_만족도_cat"].hist()
plt.show()

strat_test_set["경제상태_만족도_cat"].hist()
plt.show()

- 테스트 세트에서 만족도_경제상태 카테고리 비율을 살펴보기

In [ ]:
strat_test_set["경제상태_만족도_cat"].value_counts() / len(strat_test_set)

- 경제상태_만족도_cat 특성을 삭제해서 데이터를 원래 상태로 되돌림

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("경제상태_만족도_cat", axis=1, inplace=True)

- train_set, test_set 사이즈 계산하기

In [ ]:
print("train.size :", len(strat_train_set))
print("test.size :", len(strat_test_set))

In [ ]:
print(strat_test_set['만 60세 이후 자살 생각'].value_counts())

## (3) 머신러닝 알고리즘을  위해 데이터를 준비하기

데이터 준비 작업을 함수로 자동화하기

- def drop_features(df)
- def transform_features(df)

In [ ]:
# 결과 변수 추출

elderly = strat_train_set.drop("만 60세 이후 자살 생각", axis=1) # drop labels for training set
elderly_label = strat_train_set["만 60세 이후 자살 생각"].copy()

elderly_label

### 데이터 정제하기

- def drop_features(df)

In [ ]:
elderly.info()

- 결측치 비율 계산하기

In [ ]:
elderly.isnull().sum() / len(elderly)*100

- 결측치 존재하는 변수

1. 일상생활수행에 있어 도움받음 여부
2. 인지기능 총점
3. 작년 한 해 월평균 소비지출액(가구)
4. 부채 금액


#### 결측치 처리 1. 일상생활수행에 있어 도움받음 여부

결측치 비율이 너무 높으므로, 제거

In [ ]:
def drop_features(df):
    df = df.drop('일상생활수행에 있어 도움받음 여부', axis=1)
    return df

In [ ]:
elderly.isnull().sum() / len(elderly)*100

### 데이터 전처리 함수 만들기

- def transform_features(df)

1. (1) 수치형 변수를 처리하기
- 결측치 대체
- 표준화

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

def num_transform_simpleimpt(df):
    imputer = SimpleImputer(strategy="median")
    df_smp = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
    return df_smp

def num_transform_stdscaler(df):
    scaler = StandardScaler()
    df_std = scaler.fit_transform(df)
    df_std = pd.DataFrame(df_std, columns=df.columns)
    return df_std

1. (2) 범주형 변수를 처리하기
- 최빈값 대체
- LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

def cat_transform_simpleimpt(df):
    imputer = SimpleImputer(strategy="most_frequent")
    df_smp = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
    return df_smp

def cat_transorm_leencoder(df):
    for feature in df.columns.tolist():
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

2. 데이터 전처리 함수 만들기

In [ ]:
def transform_features(elderly):
    # 수치형 변수 데이터 프레임
    elderly_num1 = elderly.select_dtypes(include='float64')
    elderly_num2 = elderly.select_dtypes(include='int64')
    elderly_num = pd.concat([elderly_num1, elderly_num2], axis=1, join='inner')

    elderly_cat = elderly.select_dtypes(include='category')

    elderly_cat = drop_features(elderly_cat)

    df_num = num_transform_simpleimpt(elderly_num)
    df_num = num_transform_stdscaler(df_num)

    df_cat = cat_transform_simpleimpt(elderly_cat)
    df_cat = cat_transorm_leencoder(df_cat)

    df_final = pd.concat([df_cat, df_num], axis=1, join='inner')

    return df_final

3. elderly_prepared 데이터프레임 만들기

In [ ]:
elderly_prepared = transform_features(elderly)

elderly_prepared

## (4) 모델을 훈련시키고, 교차 검증과 랜덤 서치를 통해 모델을 조정하기

In [ ]:
from scipy.stats import randint
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

rf = RandomForestClassifier(class_weight='balanced')

param_distribs_rf = {'n_estimators': randint(low=1, high=200),#[80, 100, 150, 200],
                     'max_features': ['auto', 'sqrt', 'log2'],
                     'max_depth' : randint(5, 21)}# [3, 5, 10, 15]}

rf_random_search = RandomizedSearchCV(estimator = rf,
                                      param_distributions = param_distribs_rf,
                                      n_iter = 100,
                                      cv = StratifiedKFold(n_splits=5),
                                      verbose=2,
                                      random_state=42,
                                      n_jobs = -1)

rf_random_search.fit(elderly_prepared, elderly_label)

- 하이퍼파라미터 튜닝을 통한 가장 좋은 파라미터 조합 도출하기

In [ ]:
rf_random_search.best_params_

- 위를 통해 최적의 모델을 만들기

In [ ]:
final_model = rf_random_search.best_estimator_
'''
X_test = strat_test_set.drop("만 60세 이후 자살 생각", axis=1)
y_test = strat_test_set["만 60세 이후 자살 생각"].copy()

# X_test = X_test.drop("일상생활수행에 있어 도움받음 여부", axis=1)'''

#final_model.fit(elderly_prepared, elderly_label)

## (5) 오차 행렬, f1 스코어, 정확도, 재현율, 정밀도, 특이도 계산하기

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, recall_score, precision_score,  plot_confusion_matrix, roc_curve, roc_auc_score
from sklearn.metrics import classification_report

X_test = strat_test_set.drop("만 60세 이후 자살 생각", axis=1)
y_test = strat_test_set["만 60세 이후 자살 생각"]

X_test_prepared = transform_features(X_test)
final_model_pred_test = final_model.predict(X_test_prepared)

final_model_confusion_test = confusion_matrix(y_test, final_model_pred_test)
print(final_model_confusion_test)
cfreport = classification_report(y_test, final_model_pred_test)
print(cfreport)

print('F1:',f1_score(y_test, final_model_pred_test, pos_label = '예(있다)'))
print("정확도:", accuracy_score(y_test, final_model_pred_test))
print("재현율:", recall_score(y_test, final_model_pred_test, pos_label = '예(있다)'))
print("정밀도:", precision_score(y_test, final_model_pred_test, pos_label = '예(있다)'))
print("특이도:", final_model_confusion_test[0][0] / (final_model_confusion_test[0][0] + final_model_confusion_test[0][1]))

- 오차행렬 시각화하기

In [ ]:
plt.figure(figsize =(30, 30))
plot_confusion_matrix(final_model,
                      X_test_prepared, y_test,
                      include_values = True,
                      display_labels = ['아니요(없다)', '예(있다)'], # 목표변수 이름
                      cmap = 'Pastel1') # 컬러맵
plt.savefig('confusion_matrix.png') # 오차행렬 자료 저장
plt.show() # 따로 그리기

In [ ]:
print(y_test.value_counts())
print(strat_test_set['만 60세 이후 자살 생각'].value_counts())

## (6) 변수중요도 산출하기

In [ ]:
dt_importance = pd.DataFrame()
dt_importance['Feature'] = list(elderly_prepared.columns) # 설명변수 이름

dt_importance['Importance'] = final_model.feature_importances_ # 설명변수 중요도 산출
'''important = pd.DataFrame()
important['Importance'] = final_model.feature_importances_

display(important)'''

# 변수 중요도 내림차순 정렬
dt_importance.sort_values("Importance", ascending = False, inplace = True)
dt_importance.round(8)

- 변수 중요도 시각화

In [ ]:
# 변수 중요도 오름차순 정렬
dt_importance.sort_values("Importance", ascending = True, inplace = True)
coordinates = range(len(dt_importance)) # 설명변수 개수만큼 bar 시각화
plt.figure(figsize =(8, 12))
plt.barh(y = coordinates, width = dt_importance["Importance"], align='center')
plt.yticks(coordinates, dt_importance["Feature"]) # y축 눈금별 설명변수 이름 기입
plt.xlabel("Feature Importance") # x축 이름
plt.ylabel("Features") # y축 이름
plt.tight_layout()
plt.savefig('feature_importance.png') # 변수 중요도 그래프 저장
plt.show()

## (7) ROC-AUC 커브 그리고, AUC 점수 계산하기

- ROC-AUC 커브 그리기

In [ ]:
y_test = (y_test == '예(있다)').astype(int)
print(y_test.value_counts())

prob = final_model.predict_proba(X_test_prepared)
prob = prob[:, 1]
fper, tper, thresholds = roc_curve(y_test, prob)
plt.plot(fper, tper, color='red', label='ROC')
plt.plot([0, 1], [0, 1], color='green', linestyle='--')
plt.xlabel('False Positive Rate( 1 - Sensitivity )')
plt.ylabel('True Positive Rate( Recall )')
plt.title('Receiver Operating Characteristic Curve')
plt.legend()
plt.savefig('ROC_curve.png')
plt.show()

- AUC 점수 계산하기

In [ ]:
pred_proba = final_model.predict_proba(X_test_prepared)[:, 1]
auc = roc_auc_score(y_test, pred_proba)
print('AUC 값: {0:.4f}'.format(auc))